In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [ ]:
filename = 'notMNIST_reformat.pickle';
with open(filename, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
image_size = 28
num_labels = 10

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_dropout_rate = tf.placeholder(tf.float32)
  tf_train_dropout = tf.nn.dropout(tf_train_dataset, tf_dropout_rate)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dropout, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 801
dropout_rate = 0.2

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    feed_dict = { tf_dropout_rate : 1-dropout_rate }
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [ ]:
# Now that we have observed the model for a single layer of stochastic gradient descent we will 
# work on a multilayer model

batch_size = 128
h1_nodes = 1024
h2_nodes = 512
h3_nodes = 256

def multilayer_perceptron(x):
  layer_1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(x, weights['W_1']), biases['b_1'])), 1-(1-tf_keep_rate)/4)
  layer_2 = tf.nn.dropout(tf.nn.elu(tf.add(tf.matmul(layer_1, weights['W_2']), biases['b_2'])), 1-(1-tf_keep_rate)/8)
  layer_3 = tf.nn.dropout(tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['W_3']), biases['b_3'])), 1-(1-tf_keep_rate)/16)
  layer_4 = tf.matmul(layer_3, weights['W_4']) + biases['b_4']
  return layer_4

def no_dropout_perceptron(x):
  layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['W_1']), biases['b_1']))
  layer_2 = tf.nn.elu(tf.add(tf.matmul(layer_1, weights['W_2']), biases['b_2']))
  layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['W_3']), biases['b_3']))
  layer_4 = tf.matmul(layer_3, weights['W_4']) + biases['b_4']
  return layer_4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_rate = tf.placeholder(tf.float32)
  tf_train_dropout = tf.nn.dropout(tf_train_dataset, 1-(1-tf_keep_rate)/2)
  
  
  # Variables.
  weights = {
    'W_1' : tf.Variable(tf.truncated_normal([image_size * image_size, h1_nodes])),
    'W_2' : tf.Variable(tf.truncated_normal([h1_nodes, h2_nodes])),
    'W_3' : tf.Variable(tf.truncated_normal([h2_nodes, h3_nodes])),
    'W_4' : tf.Variable(tf.truncated_normal([h3_nodes, num_labels]))
  }
  
  biases = {
    'b_1' : tf.Variable(tf.zeros([h1_nodes])),
    'b_2' : tf.Variable(tf.zeros([h2_nodes])),
    'b_3' : tf.Variable(tf.zeros([h3_nodes])),
    'b_4' : tf.Variable(tf.zeros([num_labels]))
      
  }

  
  # Training computation.
  logits = multilayer_perceptron(tf_train_dropout)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.005*(tf.nn.l2_loss(weights['W_1']) + tf.nn.l2_loss(biases['b_1'])) + 0.005*(tf.nn.l2_loss(weights['W_2']) + tf.nn.l2_loss(biases['b_2'])) + 0.005*(tf.nn.l2_loss(weights['W_3']) + tf.nn.l2_loss(biases['b_3'])) + 0.005*(tf.nn.l2_loss(weights['W_4']) + tf.nn.l2_loss(biases['b_4']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(no_dropout_perceptron(tf_train_dataset))
  valid_prediction = tf.nn.softmax(no_dropout_perceptron(tf_valid_dataset))
  test_prediction = tf.nn.softmax(no_dropout_perceptron(tf_test_dataset))

In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

num_steps = 3001
dropout_rate = 0.5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    random_batch, random_labels = randomize(train_dataset, train_labels)
    # Generate a minibatch.
    batch_data = random_batch[:batch_size, :]
    batch_labels = random_labels[:batch_size, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_rate : 1-dropout_rate}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))